In [5]:
import math
import random

def distance(p1, p2):
    return math.hypot(p1[0] - p2[0], p1[1] - p2[1])

def total_cost(tour, coords):
    return sum(distance(coords[tour[i]], coords[tour[(i + 1) % len(tour)]]) for i in range(len(tour)))

def nearest_neighbor(coords):
    n = len(coords)
    unvisited = set(range(1, n))
    tour = [0]
    while unvisited:
        last = tour[-1]
        next_city = min(unvisited, key=lambda city: distance(coords[last], coords[city]))
        tour.append(next_city)
        unvisited.remove(next_city)
    return tour

def two_opt(tour, coords):
    improved = True
    while improved:
        improved = False
        for i in range(1, len(tour) - 2):
            for j in range(i + 1, len(tour)):
                if j - i == 1: continue
                new_tour = tour[:i] + tour[i:j][::-1] + tour[j:]
                if total_cost(new_tour, coords) < total_cost(tour, coords):
                    tour = new_tour
                    improved = True
    return tour

def simulated_annealing(coords, initial_tour, T=15000.0, alpha=0.9995, stopping_T=1e-8):
    current = initial_tour[:]
    best = current[:]
    best_cost = total_cost(best, coords)

    while T > stopping_T:
        a, b = sorted(random.sample(range(len(coords)), 2))
        new = current[:]
        if b - a > 1:
            new[a:b] = reversed(current[a:b])  # segment reversal (2-opt benzeri)
        else:
            new[a], new[b] = new[b], new[a]  # fallback swap

        current_cost = total_cost(current, coords)
        new_cost = total_cost(new, coords)

        if new_cost < current_cost or random.random() < math.exp((current_cost - new_cost) / T):
            current = new[:]
            if new_cost < best_cost:
                best = new[:]
                best_cost = new_cost

        T *= alpha
    return best, best_cost

def best_initial(coords, tries=10):
    best_tour = None
    best_cost = float('inf')
    for i in range(tries):
        tour = nearest_neighbor(coords)
        tour = two_opt(tour, coords)
        cost = total_cost(tour, coords)
        if cost < best_cost:
            best_tour = tour
            best_cost = cost
    return best_tour

def run_tsp_with_start(coords, initial_tour, runs=1000):
    best_overall_tour = initial_tour
    best_overall_cost = total_cost(initial_tour, coords)

    for i in range(runs):
        tour, cost = simulated_annealing(coords, initial_tour)
        if cost < best_overall_cost:
            best_overall_tour = tour
            best_overall_cost = cost
        if i % 100 == 0:
            print(f"🔁 {i}. tekrar - En iyi maliyet: {round(best_overall_cost, 2)}")

    return best_overall_tour, best_overall_cost

def read_tsp_file(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
        coords = [tuple(map(float, line.strip().split())) for line in lines[1:]]
    return coords

def save_results(filename, size, cost, path):
    with open(filename, 'a') as f:
        f.write(f"Dosya Boyutu {size}:\n")
        f.write(f"Optimal maliyet değeri: {round(cost, 2)}\n")
        f.write("Optimal maliyeti sağlayan path: " + " -> ".join(map(str, path)) + "\n\n")

# --- 318'lik veri için çalıştırma ---
filename = r'C:\Users\ALPEREN\Desktop\Okul\Algoritma Analizi ve Tasarımı\TSP_222805007_Alperen_Mengünoğul\tsp_318_2.txt'  # dosya adını kontrol et
coords = read_tsp_file(filename)

print("🔍 Başlangıç turu oluşturuluyor...")
initial = best_initial(coords, tries=10)

print("🚀 20 tekrar başlatılıyor...")
best_path, best_cost = run_tsp_with_start(coords, initial, runs=20)

print("\n🎯 Nihai sonuç:")
print("Optimal maliyet değeri:", round(best_cost, 2))
print("Optimal path:", " -> ".join(map(str, best_path)))

# Sonucu dosyaya kaydet
save_results("results.txt", 318, best_cost, best_path)



🔍 Başlangıç turu oluşturuluyor...
🚀 20 tekrar başlatılıyor...
🔁 0. tekrar - En iyi maliyet: 46821.27

🎯 Nihai sonuç:
Optimal maliyet değeri: 46821.27
Optimal path: 0 -> 313 -> 311 -> 308 -> 303 -> 192 -> 193 -> 197 -> 196 -> 312 -> 1 -> 4 -> 50 -> 2 -> 317 -> 314 -> 310 -> 307 -> 49 -> 304 -> 305 -> 306 -> 309 -> 315 -> 316 -> 3 -> 7 -> 8 -> 15 -> 14 -> 19 -> 20 -> 26 -> 25 -> 21 -> 18 -> 9 -> 17 -> 22 -> 24 -> 31 -> 38 -> 43 -> 44 -> 34 -> 35 -> 33 -> 32 -> 141 -> 142 -> 39 -> 40 -> 45 -> 152 -> 151 -> 147 -> 146 -> 143 -> 148 -> 149 -> 150 -> 145 -> 144 -> 138 -> 137 -> 116 -> 115 -> 111 -> 110 -> 158 -> 89 -> 88 -> 84 -> 63 -> 54 -> 55 -> 64 -> 85 -> 92 -> 87 -> 86 -> 154 -> 95 -> 98 -> 100 -> 105 -> 106 -> 109 -> 155 -> 107 -> 104 -> 101 -> 97 -> 94 -> 91 -> 93 -> 96 -> 102 -> 103 -> 108 -> 112 -> 113 -> 120 -> 119 -> 124 -> 125 -> 131 -> 140 -> 139 -> 130 -> 126 -> 123 -> 136 -> 134 -> 135 -> 133 -> 132 -> 128 -> 121 -> 118 -> 117 -> 129 -> 127 -> 122 -> 114 -> 10 -> 11 -> 6 -> 5 